# RRTMGP_RFMIP_LW_02

*Last edited: 2024-10-18*

This code is part of RRTM for GCM Applications - Parallel (RRTMGP). Please see `rrtmgp_rfmip_lw.F90` for more info.

- Example program to demonstrate the calculation of longwave radiative fluxes in clear, aerosol-free skies.
- The example files come from the Radiative Forcing MIP (https://www.earthsystemcog.org/projects/rfmip/).
- The large problem (1800 profiles) is divided into blocks.
- Program is invoked as rrtmgp_rfmip_lw [block_size input_file coefficient_file upflux_file downflux_file] .
- All arguments are optional but need to be specified in order.

The objective of this Notebook is to study the functioning of the example program `rrtmgp_rfmip_lw.F90`, taking advantage of Jupyter's resources for code documentation and interactive experimentation.

For ease of understanding, this version does not use OpenACC, OpenMP, Timing, Papi, and other optional libraries.

## Table of Contents

> (in JupyterLab, use the menu, or Ctrl+Shift+K)

## Makefile

For `rrtmgp_rfmip_lw_02.f90` and `rrtmgp_rfmip_sw_02.F90`

In [242]:
#23456789 123456789 123456789 123456789 123456789 123456789 123456789 12

In [243]:
WORKDIR = "/home/x/git/pd1b24/rrtmgp/ukk23test01/examples/rfmip-clear-sky/"
%cd {WORKDIR}

/home/x/git/pd1b24/rrtmgp/ukk23test01/examples/rfmip-clear-sky


In [244]:
%%writefile Makefile
# Set k-distribution files
# KDIST_LW=rrtmgp-data-lw-g256-2018-12-04.nc
# KDIST_SW=rrtmgp-data-sw-g224-2018-12-04.nc
KDIST_LW=rrtmgp-data-lw-g128-210809.nc
KDIST_SW=rrtmgp-data-sw-g112-210809.nc
BLOCKSIZE = 36

# Here set variables RRTMGP_BUILD, NCHOME, NFHOME, TIME_DIR (for GPTL)
# and link against BLAS library, or have those variables set in the
# environment

RRTMGP_ROOT=../../
$(info VAR="$(RRTMGP_ROOT)")
RRTMGP_BUILD=$(RRTMGP_ROOT)/build
-include $(RRTMGP_BUILD)/Makefile.conf

# RRTMGP library, module files

LDFLAGS   += -L$(RRTMGP_BUILD)
LIBS      += -lrrtmgp -lrte -lneural
FCINCLUDE += -I$(RRTMGP_BUILD)

# netcdf library, module files
# C and Fortran interfaces respectively

FCINCLUDE += -I$(NFHOME)/include
LDFLAGS   += -L$(NFHOME)/lib -L$(NCHOME)/lib
LIBS      += -lnetcdff -lnetcdf

# -------- BLAS lbirary: required for RRTMGP-NN -------- 
# Choose BLAS library: e.g. "blis", "openblas", "mkl".
# Here we use BLIS (https://github.com/flame/blis) which is one of the
# fastest BLAS libraries available (on Intel platforms MKL is probably
# faster). The flags BLAS_DIR and BLASLIB can be specified in
# Makefile.conf or the environment Depending on your BLAS library you
# might have to make changes below.

ifeq ($(BLASLIB),blis)
	FCINCLUDE   += -I$(BLAS_DIR)/include/blis
	LIBS      	+= $(BLAS_DIR)/lib/libblis.a -lm -lpthread 
else ifeq ($(BLASLIB),blis-amd)
	FCINCLUDE   += -I$(BLAS_DIR)/include/blis
	LIBS      	+= $(BLAS_DIR)/lib/libblis-mt.a -lm -lpthread 
else ifeq ($(BLASLIB),openblas)
	LIBS 		+= -lopenblas 
else ifeq ($(BLASLIB),mkl)
	FCINCLUDE   += -I${MKLROOT}/include
	#LDFLAGS     += -L${MKLROOT}/lib/intel64 -lmkl_rt
	#LDFLAGS     += -lpthread -lm -ldl
endif

# -------- Use OpenMP? May be needed for BLAS -------
# ifeq ($(BLASLIB),blis-amd)
#    USE_OPENMP=1
# endif
#USE_OPENMP=1

ifeq ($(USE_OPENMP),1)
	# OMPFLAG = -fopenmp
	FCFLAGS += -DUSE_OPENMP
endif
OMPFLAG = -fopenmp

ifeq ($(USE_OPENACC),1)
  FCFLAGS += -DUSE_OPENACC
endif

# -------- GPTL Timing -------
# Use General Purpose Timing Library to profile code?

ifeq ($(GPTL_TIMING),1)

	# Timing library

	FCINCLUDE += -I$(TIME_DIR)/include

    # Compiler specific

	FCFLAGS += -DUSE_TIMING
	LDFLAGS   += -L$(TIME_DIR)/lib -Wl,-rpath=$(TIME_DIR)/lib
	LIBS      += -lgptl

# Use GPTL together with PAPI hardware instrumentation to measure 
# computational intensity? Requires that your GPTL was compiled to PAPI 
# support

else ifeq ($(GPTL_TIMING),2)

	# Timing library

	FCINCLUDE += -I$(TIME_DIR)/include

	# Compiler specific

	FCFLAGS += -DUSE_TIMING -DUSE_PAPI
	LDFLAGS   += -L$(TIME_DIR)/lib -Wl,-rpath=$(TIME_DIR)/lib

	# LIBS      += -lgptl  -rdynamic  -lpapi

	LIBS      += -lgptl -lpapi

    ifeq ($(FC),gfortran)
    	LIBS      += -rdynamic
    endif

endif

VPATH = ../

# Compilation rules
%.o: %.F90
	$(FC) $(FCFLAGS) $(FCINCLUDE) -c $< $(OMPFLAG)

%: %.o
	$(FC) $(FCFLAGS) -o $@ $^ $(LDFLAGS) $(LIBS) $(OMPFLAG)

#
# Ancillary codes
#
ADDITIONS = mo_simple_netcdf.o mo_rfmip_io.o mo_load_coefficients.o
all: rrtmgp_rfmip_lw_02 jn_rrtmgp_rfmip_sw
rrtmgp_rfmip_lw_02:   rrtmgp_rfmip_lw_02.o   $(ADDITIONS) $(RRTMGP_BUILD)/librte.a $(RRTMGP_BUILD)/librrtmgp.a $(RRTMGP_BUILD)/libneural.a
rrtmgp_rfmip_lw_02.o: rrtmgp_rfmip_lw_02.f90 $(ADDITIONS)
jn_rrtmgp_rfmip_sw:   jn_rrtmgp_rfmip_sw.o   $(ADDITIONS) $(RRTMGP_BUILD)/librte.a $(RRTMGP_BUILD)/librrtmgp.a $(RRTMGP_BUILD)/libneural.a
jn_rrtmgp_rfmip_sw.o: jn_rrtmgp_rfmip_sw.F90 $(ADDITIONS)
mo_rfmip_io.o:          mo_rfmip_io.F90          mo_simple_netcdf.o
mo_load_coefficients.o: mo_load_coefficients.F90 mo_simple_netcdf.o

tests: multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc # Also the template files
	# Files need to have been generated/downloaded before
	$(RUN_CMD) ./rrtmgp_rfmip_lw_02 $(BLOCKSIZE) multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ${RRTMGP_ROOT}/rrtmgp/data/$(KDIST_LW)
	$(RUN_CMD) ./jn_rrtmgp_rfmip_sw $(BLOCKSIZE) multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ${RRTMGP_ROOT}/rrtmgp/data/$(KDIST_SW)

check:
	cd ${RRTMGP_ROOT}/examples/rfmip-clear-sky && python ./compare-to-reference.py --fail=7.e-4

multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc:
	bash ./stage_files.sh

clean:
	-rm jn_rrtmgp_rfmip_sw rrtmgp_rfmip_lw_02 *.o *.mod *.optrpt

Overwriting Makefile


## rrtmgp_rfmip_lw_02.f90

All cells need to be executed in order to create the final file rrtmgp_rfmip_lw_02.f90. The code has been divided in this way, into small cells, to try to make the parts easier to understand.

Start of the code

- Error checking: Procedures in rte+rrtmgp return strings which are empty if no errors occured.
- Check the incoming string, print it out and stop execution if non-empty.

In [245]:
%%writefile rrtmgp_rfmip_lw_02.f90
! *** rrtmgp_rfmip_lw_02.f90 ***
! This file was generated by the Notebook `rrtmgp_rfmip_lw_02.ipynb`, 
! and it is rewritten each time the Notebook is run, losing all changes 
! made to this file. To make changes to the file, you must make them 
! directly in the Notebook, and rerun it to recreate this file with the 
! changes included.

Overwriting jn_rrtmgp_rfmip_lw.f90


### Subroutines

In [246]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

subroutine stop_on_err(error_msg)
  use iso_fortran_env, only : error_unit
  use iso_c_binding
  character(len=*), intent(in) :: error_msg

  if(error_msg /= "") then
    write (error_unit,*) trim(error_msg)
    write (error_unit,*) "Stopping."
    stop
  end if
end subroutine stop_on_err

Appending to jn_rrtmgp_rfmip_lw.f90


### Main program

In [247]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

program rrtmgp_rfmip_lw_02
implicit none

Appending to jn_rrtmgp_rfmip_lw.f90


### Modules for working with rte and rrtmgp

 Working precision for real variables:

In [248]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_rte_kind, only: wp, sp, wl, i4

Appending to jn_rrtmgp_rfmip_lw.f90


- Optical properties of the atmosphere as array of values
  - In the longwave we include only absorption optical depth (_1scl)
  - Shortwave calculations would use optical depth, single-scattering albedo, asymmetry parameter (_2str)

In [249]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_optical_props, only: ty_optical_props_1scl

Appending to jn_rrtmgp_rfmip_lw.f90


- Gas optics: maps physical state of the atmosphere to optical properties

In [250]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_gas_optics_rrtmgp, only: ty_gas_optics_rrtmgp

Appending to jn_rrtmgp_rfmip_lw.f90


- Gas optics uses a derived type to represent gas concentrations compactly...

In [251]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

 use mo_gas_concentrations, only: ty_gas_concs

Appending to jn_rrtmgp_rfmip_lw.f90


- ... and another type to encapsulate the longwave source functions.

In [252]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_source_functions, only: ty_source_func_lw

Appending to jn_rrtmgp_rfmip_lw.f90


- RTE longwave driver.

In [253]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_rte_lw, only: rte_lw

Appending to jn_rrtmgp_rfmip_lw.f90


- RTE driver uses a derived type to reduce spectral fluxes to whatever the user wants
  - Here we're just reporting broadband fluxes

In [254]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_fluxes, only: ty_fluxes_broadband, ty_fluxes_flexible

Appending to jn_rrtmgp_rfmip_lw.f90


  - modules for reading and writing files
  - RRTMGP's gas optics class needs to be initialized with data read from a netCDF files

In [255]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use mo_load_coefficients, only: load_and_init
use mo_rfmip_io, only: read_size, read_and_block_pt, &
  read_and_block_gases_ty, unblock_and_write, unblock, &
  read_and_block_lw_bc, determine_gas_names
use mo_simple_netcdf, only: read_field, get_dim_size
use netcdf
use mod_network_rrtmgp

Appending to jn_rrtmgp_rfmip_lw.f90


Local variables:

In [256]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

character(len=132) :: rfmip_file = &
  'multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc', &
  kdist_file = 'coefficients_lw.nc'

Appending to jn_rrtmgp_rfmip_lw.f90


- Neural networks for gas optics (optional) - netCDF files which describe the models and pre-processing coefficients (The two models predict SW absorption and Rayleigh scattering, respectively)

In [257]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

character(len=80) :: modelfile_tau = &
  "../../neural/data/BEST_tau-lw-18-58-58.nc", &
  modelfile_source = &
  "../../neural/data/BEST_pfrac-18-16-16.nc"

Appending to jn_rrtmgp_rfmip_lw.f90


In [258]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

character(len=132) :: flx_file, flx_file_ref, flx_file_lbl, timing_file
integer            :: nargs, ncol, nlay, nbnd, ngpt, nexp, nblocks, &
                      block_size, forcing_index, physics_index, &
                      n_quad_angles = 1
logical            :: top_at_1
integer            :: b, icol, ilay, ibnd, igpt, count_rate, iTime1, &
                      iTime2, iTime3, ncid, ninputs, istat, igas, ret, i
character(len=5)   :: block_size_char, forcing_index_char = '1', &
                      physics_index_char = '1'
character(len=32), dimension(:), allocatable :: kdist_gas_names, &
                      rfmip_gas_names
! block_size, nlay, nblocks
real(wp), dimension(:,:,:), allocatable :: p_lay, p_lev, t_lay, t_lev
real(wp), dimension(:,:,:), target, allocatable :: flux_up, flux_dn
real(wp), dimension(:,:,:,:), target, allocatable :: gpt_flux_up, &
                      gpt_flux_dn
real(wp), dimension(:,:,:), allocatable :: rlu_ref, rld_ref, rlu_nn, &
                      rld_nn, rlu_lbl, rld_lbl, rldu_ref, rldu_nn, &
                      rldu_lbl, col_dry
! block_size, nblocks (emissivity is spectrally constant)
real(wp), dimension(:,:), allocatable :: sfc_emis, sfc_t
! nbands, block_size (spectrally-resolved emissivity)
real(wp), dimension(:,:), allocatable :: sfc_emis_spec
real(wp) :: bb_flux_up
logical :: use_rrtmgp_nn, do_gpt_flux, compare_flux, save_flux

Appending to jn_rrtmgp_rfmip_lw.f90


Classes used by rte+rrtmgp:

In [259]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

type(ty_gas_optics_rrtmgp)  :: k_dist
type(ty_source_func_lw)     :: source
type(ty_optical_props_1scl) :: optical_props
type(ty_fluxes_flexible)   :: fluxes

! First model is for absorption, second is for Planck fraction
type(rrtmgp_network_type), dimension(:), allocatable :: neural_nets

Appending to jn_rrtmgp_rfmip_lw.f90


- `ty_gas_concentration` holds multiple columns; we make an array of these objects to leverage what we know about the input file

In [260]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

type(ty_gas_concs), dimension(:), allocatable  :: gas_conc_array

Appending to jn_rrtmgp_rfmip_lw.f90


### Code starts

All arguments are optional

### I/O and settings

Use neural networks for gas optics?  if NN models provided, set to true, but can also be overriden:

In [261]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

use_rrtmgp_nn      = .false.

Appending to jn_rrtmgp_rfmip_lw.f90


Save fluxes:

In [262]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

save_flux    = .false.

Appending to jn_rrtmgp_rfmip_lw.f90


Compare fluxes to reference code as well as line-by-line (RFMIP only):

In [263]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

compare_flux = .false.

Appending to jn_rrtmgp_rfmip_lw.f90


Compute fluxes per g-point? :

In [264]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

do_gpt_flux = .false.

Appending to jn_rrtmgp_rfmip_lw.f90


In [265]:
# 3456789 123456789 123456789 123456789 123456789 123456789 123456789

In [266]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

print *, "Usage: "
print *, "rrtmgp_rfmip_lw_02 \"
print *, "    [block_size] \"
print *, "    [rfmip_file] \"
print *, "    [k-distribution_file] \"
print *, "    [forcing_index (1,2,3)] \"
print *, "    [physics_index (1,2)]  \"
print *, "    [input_output file]"
print *, "OR:"
print *, "rrtmgp_rfmip_lw_02 "
print *, "    [block_size] \"
print *, "    [rfmip_file] \"
print *, "    [k-distribution_file] \"
print *, "    [forcing_index] \"
print *, "    [physics_index] \"
print *, "    [NN_lw_abs_file] \"
print *, "    [NN_lw_planck_file]"

Appending to jn_rrtmgp_rfmip_lw.f90


In [267]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

nargs = command_argument_count()
call get_command_argument(1, block_size_char)
read(block_size_char, '(i5)') block_size
if(nargs >= 2) call get_command_argument(2, rfmip_file)
if(nargs >= 3) call get_command_argument(3, kdist_file)
if(nargs >= 4) call get_command_argument(4, forcing_index_char)
if(nargs >= 5) call get_command_argument(5, physics_index_char)
if(nargs >= 6) then
  use_rrtmgp_nn = .true.
  call get_command_argument(6, modelfile_tau)
  if (nargs >= 7) then
    allocate(neural_nets(2))
    call get_command_argument(7, modelfile_source)
  else 
    allocate(neural_nets(1))
  end if
end if

Appending to jn_rrtmgp_rfmip_lw.f90


In [268]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call read_size(rfmip_file, ncol, nlay, nexp)

Appending to jn_rrtmgp_rfmip_lw.f90


In [269]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (nexp==18) compare_flux=.true.

print *, "input file:", rfmip_file
print *, "nexp:", nexp, "ncol:", ncol, "nlay:", nlay, &
  "block_size:", block_size

if(mod(ncol*nexp, block_size) /= 0 ) &
  call stop_on_err("rrtmgp_rfmip_lw: number of columns doesn't fit &
    evenly into blocks.")
nblocks = (ncol*nexp)/block_size
print *, "Doing ",  nblocks, "blocks of size ", block_size

read(forcing_index_char, '(i4)') forcing_index
if(forcing_index < 1 .or. forcing_index > 4) &
  stop "Forcing index is invalid (must be 1, 2 ou 3)"

read(physics_index_char, '(i4)') physics_index
if(physics_index < 1 .or. physics_index > 2) &
  stop "Physics index is invalid (must be 1 ou 2)"

if(physics_index == 2) n_quad_angles = 3

Appending to jn_rrtmgp_rfmip_lw.f90


Save upwelling and downwelling fluxes in the same file:

In [270]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (use_rrtmgp_nn) then
  flx_file = 'output_fluxes/rlud_Efx_RTE-RRTMGP-NN-181204_rad-irf_' // &
             'r1i1p1f' // trim(forcing_index_char) // '_gn.nc'
else
  flx_file = 'output_fluxes/rlud_Efx_RTE-RRTMGP-181204_rad-irf_' // &
             'r1i1p1f' // trim(forcing_index_char) // '_gn.nc'
end if

Appending to jn_rrtmgp_rfmip_lw.f90


Identify the set of gases used in the calculation based on the forcing index

A gas might have a different name in the k-distribution than in the files provided by RFMIP (e.g. 'co2' and 'carbon_dioxide')

In [271]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call determine_gas_names(rfmip_file, kdist_file, forcing_index, &
  kdist_gas_names, rfmip_gas_names)

print *, "Calculation uses RFMIP gases: ", &
  (trim(kdist_gas_names(b)) // " ", b = 1, size(kdist_gas_names))

Appending to jn_rrtmgp_rfmip_lw.f90


### Load Neural Network models

In [272]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (use_rrtmgp_nn) then
  print *, 'loading longwave absorption model from ', modelfile_tau
  call neural_nets(1) % load_netcdf(modelfile_tau)
  if (nargs >= 7) then 
    print *, 'loading Planck fraction model from ', modelfile_source
    call neural_nets(2) % load_netcdf(modelfile_source)
  end if
  ninputs = size(neural_nets(1) % layers(1) % w_transposed, 2)
  print *, "NN supports gases: ", &
    (trim(neural_nets(1) % input_names(b)) // " ", b = 3, &
    size(neural_nets(1) % input_names))
end if

Appending to jn_rrtmgp_rfmip_lw.f90


### Prepare data for use in rte+rrtmgp

Allocation on assignment within reading routines

In [273]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call read_and_block_pt(rfmip_file, block_size, p_lay, p_lev, t_lay, &
  t_lev)
! print *, "shape t_lay, min, max", shape(t_lay), maxval(t_lay), &
  minval(t_lay)

Appending to jn_rrtmgp_rfmip_lw.f90


Are the arrays ordered in the vertical with 1 at the top or the bottom of the domain?

In [274]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

top_at_1 = p_lay(1, 1, 1) < p_lay(nlay, 1, 1)

Appending to jn_rrtmgp_rfmip_lw.f90


Read the gas concentrations and surface properties

In [275]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call read_and_block_gases_ty(rfmip_file, block_size, kdist_gas_names, &
  rfmip_gas_names, gas_conc_array)
! do b = 1, size(gas_conc_array(1)%concs)
!  print *, "max of gas ", gas_conc_array(1)%gas_name(b), ":", &
!    maxval(gas_conc_array(1)%concs(b)%conc)
! end do
call read_and_block_lw_bc(rfmip_file, block_size, sfc_emis, sfc_t)

Appending to jn_rrtmgp_rfmip_lw.f90


Read k-distribution information. load_and_init() reads data from netCDF and calls k_dist%init(); users might want to use their own reading methods

In [276]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call load_and_init(k_dist, trim(kdist_file), gas_conc_array(1))

! print *, "min of play", minval(p_lay), &
!  "p_lay = k_dist%get_press_min()", k_dist%get_press_min()
! print *," press min max", k_dist%get_press_min(), k_dist%get_press_max()
! print *," temp min max", k_dist%get_temp_min(), k_dist%get_temp_max()

where(p_lay < k_dist%get_press_min()) &
  p_lay = k_dist%get_press_min() + spacing(k_dist%get_press_min())

if(.not. k_dist%source_is_internal()) &
  stop "rrtmgp_rfmip_lw: k-distribution file isn't LW"

nbnd = k_dist%get_nband()
ngpt = k_dist%get_ngpt()

Appending to jn_rrtmgp_rfmip_lw.f90


RRTMGP won't run with pressure less than its minimum. The top level in the RFMIP file is set to 10^-3 Pa. Here we pretend the layer is just a bit less deep. This introduces an error but shows input sanitizing.

In [277]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (top_at_1) then
  p_lev(1,:,:) = k_dist%get_press_min() + epsilon(k_dist%get_press_min())
else
  p_lev(nlay+1,:,:) = k_dist%get_press_min() + &
    epsilon(k_dist%get_press_min())
end if
! print *," shape play", shape(p_lay)
! print *, "play sfc", maxval(p_lay(nlay,:,:)), "tlay sfc", &
!  maxval(t_lay(nlay,:,:))

Appending to jn_rrtmgp_rfmip_lw.f90


Allocate space for output fluxes (accessed via pointers in ty_fluxes_broadband), gas optical properties, and source functions. The %alloc() routines carry along the spectral discretization from the k-distribution.

In [278]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

allocate(flux_up(nlay+1, block_size, nblocks), &
         flux_dn(nlay+1, block_size, nblocks))

! Allocate g-point fluxes if desired
if (do_gpt_flux) then
  allocate(gpt_flux_up(ngpt, nlay+1, block_size, nblocks), &
           gpt_flux_dn(ngpt, nlay+1, block_size, nblocks))
end if

allocate(sfc_emis_spec(nbnd, block_size))
!$acc enter data create(sfc_emis_spec) copyin(sfc_emis)

! OpenACC: Arrays are allocated on device inside constructor
call stop_on_err(source%alloc(block_size, nlay, k_dist))
call stop_on_err(optical_props%alloc_1scl(block_size, nlay, k_dist))

Appending to jn_rrtmgp_rfmip_lw.f90


In [279]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (use_rrtmgp_nn) then
  print *, "starting clear-sky LW computations, using neural &
           networks as RRTMGP kernel"
else
  print *, "starting clear-sky LW computations, using &
           lookup-table as RRTMGP kernel"
end if
call system_clock(count_rate=count_rate)
call system_clock(iTime1)

Appending to jn_rrtmgp_rfmip_lw.f90


---

### Loop over blocks

In [280]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

do b = 1, nblocks
  fluxes%flux_up => flux_up(:,:,b)
  fluxes%flux_dn => flux_dn(:,:,b)    
  if (do_gpt_flux) then
    fluxes%gpt_flux_up => gpt_flux_up(:,:,:,b)
    fluxes%gpt_flux_dn => gpt_flux_dn(:,:,:,b)
  end if

Appending to jn_rrtmgp_rfmip_lw.f90


Expand the spectrally-constant surface emissivity to a per-band emissivity for each column 

In [281]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

do icol = 1, block_size
  do ibnd = 1, nbnd
    sfc_emis_spec(ibnd,icol) = sfc_emis(icol,b)
  end do
end do

Appending to jn_rrtmgp_rfmip_lw.f90


Compute the optical properties of the atmosphere and the Planck source functions from pressures, temperatures, and gas concentrations...

In [282]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (use_rrtmgp_nn) then
  call stop_on_err(k_dist%gas_optics(p_lay(:,:,b),      &
                                    p_lev(:,:,b),       &
                                    t_lay(:,:,b),       &
                                    sfc_t(:,b),         &
                                    gas_conc_array(b),  &
                                    optical_props,      &
                                    source,             &
                                    tlev = t_lev(:,:,b),&
                                    neural_nets = neural_nets))
else
  call stop_on_err(k_dist%gas_optics(p_lay(:,:,b),      &
                                    p_lev(:,:,b),       &
                                    t_lay(:,:,b),       &
                                    sfc_t(:,b),         &
                                    gas_conc_array(b),  &
                                    optical_props,      &
                                    source,             &
                                    tlev = t_lev(:,:,b)))
end if

! print *, "mean of pfrac is:", mean_3d(planck_frac(:,:,:,b))
! print *, "mean of tau is:", mean_3d(optical_props%tau)
! print *, "mean of lay_source is:", mean_3d(source%lay_source)
! print *, "mean of lev_source is:", mean_3d(source%lev_source)

Appending to jn_rrtmgp_rfmip_lw.f90


In [283]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call system_clock(iTime2)

Appending to jn_rrtmgp_rfmip_lw.f90


... and compute the spectrally-resolved fluxes, providing reduced values via ty_fluxes_broadband

In [284]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call stop_on_err(rte_lw(optical_props, & 
                        top_at_1, & 
                        source, & 
                        sfc_emis_spec, & 
                        fluxes, & 
                        n_gauss_angles = n_quad_angles, & 
                        use_2stream = .false.))


Appending to jn_rrtmgp_rfmip_lw.f90


In [285]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

end do ! blocks

Appending to jn_rrtmgp_rfmip_lw.f90


**End of block**

---

In [286]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

call system_clock(iTime3)

Appending to jn_rrtmgp_rfmip_lw.f90


In [287]:
#23456789 123456789 123456789 123456789 123456789 123456789 123456789 12

In [288]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (nblocks==1) then
  print *, "-----------------------------------------------------------"
  print '(a,f11.4,/,a,f11.4,/,a,f11.4,a)', &
    ' Time elapsed in gas optics:', &
    real(iTime2-iTime1) / real(count_rate), &
    ' Time elapsed in solver:    ', &
    real(iTime3-iTime2) / real(count_rate), &
    ' Time elapsed in total:     ', &
    real(iTime3-iTime1) / real(count_rate)
  print *, "-----------------------------------------------------------"
else
  print *,'Elapsed time on everything ', &
    real(iTime3-iTime1) / real(count_rate)
end if

Appending to jn_rrtmgp_rfmip_lw.f90


In [289]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

! Also deallocates arrays on device
call optical_props%finalize()

! Also deallocates arrays on device
call source%finalize()

print*,"-----------------------------------------------------------"//&
       "-----------------------------------------------------------"

! mean of flux_down is: 103.2458
print *, "mean of flux_down is:", mean_3d(flux_dn)

print *, "mean of flux_up is:", mean_3d(flux_up)


Appending to jn_rrtmgp_rfmip_lw.f90


Save fluxes?

In [290]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (save_flux) then
  print *, "Attempting to save fluxes to ", flx_file
  call unblock_and_write(trim(flx_file), 'rlu', flux_up)
  call unblock_and_write(trim(flx_file), 'rld', flux_dn)
  print *, "Fluxes saved to ", flx_file
end if 

Appending to jn_rrtmgp_rfmip_lw.f90


Compare fluxes to benchmark line-by-line results, alongside reference RTE+RRTMGP computations?

In [291]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

if (compare_flux) then
  print *, "------------------------------------------------------" //&
          "------------------------------------------------------"
  print *, "-----COMPARING ERRORS (W.R.T. LINE-BY-LINE) OF NEW " //&
          "RESULTS AND RRTMGP-256  -------"
  print *, "------------------------------------------------------" //&
          "------------------------------------------------------"

  allocate(rld_ref(nlay+1, ncol, nexp))
  allocate(rlu_ref(nlay+1, ncol, nexp))
  allocate(rldu_ref(nlay+1, ncol, nexp))
  allocate(rld_nn(nlay+1, ncol, nexp))
  allocate(rlu_nn(nlay+1, ncol, nexp))
  allocate(rldu_nn(nlay+1, ncol, nexp))
  allocate(rld_lbl(nlay+1, ncol, nexp))
  allocate(rlu_lbl(nlay+1, ncol, nexp))
  allocate(rldu_lbl(nlay+1, ncol, nexp))

  flx_file_ref = 'output_fluxes/rlud_Efx_RTE-RRTMGP-181204_rad-' //&
                 'irf_r1i1p1f1_gn_REF-DP.nc'
  flx_file_lbl = 'output_fluxes/rlud_Efx_LBLRTM-12-8_rad-irf_' //&
                 'r1i1p1f1_gn.nc'

  call unblock(flux_up, rlu_nn)
  call unblock(flux_dn, rld_nn)
  rldu_nn = rld_nn - rlu_nn

  if(nf90_open(trim(flx_file_ref), NF90_NOWRITE, ncid) /= NF90_NOERR) &
    call stop_on_err("read_and_block_gases_ty: can't find file " //&
                     trim(flx_file_ref))
  rlu_ref = read_field(ncid, "rlu", nlay+1, ncol, nexp)
  rld_ref = read_field(ncid, "rld", nlay+1, ncol, nexp)
  rldu_ref = rld_ref - rlu_ref

  if(nf90_open(trim(flx_file_lbl), NF90_NOWRITE, ncid) /= NF90_NOERR) &
    call stop_on_err("read_and_block_gases_ty: can't find file " //&
                     trim(flx_file_lbl))
  rlu_lbl = read_field(ncid, "rlu", nlay+1, ncol, nexp)
  rld_lbl = read_field(ncid, "rld", nlay+1, ncol, nexp)
  rldu_lbl = rld_lbl - rlu_lbl


Appending to jn_rrtmgp_rfmip_lw.f90


In [292]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

print *, "------------- UPWELLING --------------"

print *, "MAE in upwelling fluxes of new result and RRTMGP, " //&
         "present-day:", &
         mae(reshape(rlu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rlu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))

print *, "MAE in upwelling fluxes of new result and RRTMGP, " //&
         "future:", &
         mae(reshape(rlu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rlu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
         reshape(rlu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))

! bias in upwelling flux of new result and RRTMGP,
! present-day, top-of-atm.
print *, "bias in upwelling flux of new result and RRTMGP, " //&
         "present-day, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,1), shape = [1*ncol]), &
         reshape(rlu_nn(1,:,1), shape = [1*ncol])), &
         bias(reshape(rlu_lbl(1,:,1), shape = [1*ncol]), &
         reshape(rlu_ref(1,:,1), shape = [1*ncol]))

! bias in upwelling flux of new result and RRTMGP,
! future, top-of-atm.
print *, "bias in upwelling flux of new result and RRTMGP, " //&
         "future, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,4), shape = [1*ncol]), &
         reshape(rlu_nn(1,:,4), shape = [1*ncol])), &
         bias(reshape(rlu_lbl(1,:,4), shape = [1*ncol]), &
         reshape(rlu_ref(1,:,4), shape = [1*ncol]))

! bias in upwelling flux of new result and RRTMGP,
! ALL EXPS, top-of-atm.
print *, "bias in upwelling flux of new result and RRTMGP, " //&
         "ALL EXPS, top-of-atm.:", &
         bias(reshape(rlu_lbl(1,:,:), shape = [nexp*ncol]), &
         reshape(rlu_nn(1,:,:), shape = [nexp*ncol])), &
         bias(reshape(rlu_lbl(1,:,:), shape = [nexp*ncol]), &
         reshape(rlu_ref(1,:,:), shape = [nexp*ncol]))


SyntaxError: invalid syntax (2058159830.py, line 1)

In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

print *, "-------------- DOWNWELLING --------------"

print *, "MAE in downwelling fluxes of new result and RRTMGP, " //&
         "present-day:", &
         mae(reshape(rld_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rld_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
         mae(reshape(rld_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
         reshape(rld_ref(:,:,1), shape = [1*ncol*(nlay+1)]))

! print *, "MAE in downwelling fluxes of new result and RRTMGP, " //&
!          "future:", &
!          mae(reshape(rld_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!          reshape(rld_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!          mae(reshape(rld_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!          reshape(rld_ref(:,:,4), shape = [1*ncol*(nlay+1)]))

print *, "-------------- NET FLUX --------------"

print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " //&
         "pres.day:", &
         maxval(abs(rldu_lbl(:,:,1) - rldu_nn(:,:,1))), &
         maxval(abs(rldu_lbl(:,:,1) - rldu_ref(:,:,1)))

! print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " //&
!          "future:", &
!          maxval(abs(rldu_lbl(:,:,4) - rldu_nn(:,:,4))), &
!          maxval(abs(rldu_lbl(:,:,4) - rldu_ref(:,:,4)))

print *, "Max-vertical-error in net fluxes of new result and RRTMGP, " //&
         "future-all:", &
         maxval(abs(rldu_lbl(:,:,17) - rldu_nn(:,:,17))), &
         maxval(abs(rldu_lbl(:,:,17) - rldu_ref(:,:,17)))


In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

! Print header
print *, "---------"
! Print MAE for present-day net fluxes
print *, "MAE in net fluxes of new result and RRTMGP, present-day: ", &
        mae(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
        reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
        mae(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
        reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))
! Print MAE for future net fluxes
!print *, "#MAE in net fluxes of new result and RRTMGP, future: ", &
!        mae(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!        mae(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))
! Print MAE for future-all net fluxes
print *, "MAE in net fluxes of new result and RRTMGP, future-all: ", &
        mae(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
        reshape(rldu_nn(:,:,17), shape = [1*ncol*(nlay+1)])), &
        mae(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
        reshape(rldu_ref(:,:,17), shape = [1*ncol*(nlay+1)]))
! Print MAE for all experiments
print *, "MAE in net fluxes of new result and RRTMGP, ALL EXPS: ", &
        mae(reshape(rldu_lbl(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
        reshape(rldu_nn(:,:,:), shape = [nexp*ncol*(nlay+1)])), &
        mae(reshape(rldu_lbl(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
        reshape(rldu_ref(:,:,:), shape = [nexp*ncol*(nlay+1)]))

print *, "---------"

! Print RMSE for present-day net fluxes
!print *, "#RMSE in net fluxes of new result and RRTMGP, present-day: ", &
!        rmse(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
!        rmse(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))

! Print RMSE for present-day net fluxes, surface
!print *, "RMSE in net fluxes of new result and RRTMGP, present-day, SURFACE: ", &
!        rmse(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rldu_nn(nlay+1,:,1), shape = [1*ncol])), &
!        rmse(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rldu_ref(nlay+1,:,1), shape = [1*ncol]))

! Print RMSE for present-day net fluxes, TOA
!print *, "#RMSE in net fluxes of new result and RRTMGP, present-day, TOA: ", &
!        rmse(reshape(rldu_lbl(1,:,1), shape = [1*ncol]), &
!        reshape(rldu_nn(1,:,1), shape = [1*ncol])), &
!        rmse(reshape(rldu_lbl(1,:,1), shape = [1*ncol]), &
!        reshape(rldu_ref(1,:,1), shape = [1*ncol]))

! Print RMSE for future-all net fluxes, surface
!print *, "#RMSE in net fluxes of new result and RRTMGP, future-all, SURFACE: ", &
!        rmse(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
!        reshape(rldu_nn(nlay+1,:,17), shape = [1*ncol])), &
!        rmse(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
!        reshape(rldu_ref(nlay+1,:,17), shape = [1*ncol]))

! Print RMSE for pre-industrial net fluxes, surface
!print *, "#RMSE in net fluxes of new result and RRTMGP, pre-industrial, SURFACE: ", &
!        rmse(reshape(rldu_lbl(nlay+1,:,2), shape = [1*ncol]), &
!        reshape(rldu_nn(nlay+1,:,2), shape = [1*ncol])), &
!        rmse(reshape(rldu_lbl(nlay+1,:,2), shape = [1*ncol]), &
!        reshape(rldu_ref(nlay+1,:,2), shape = [1*ncol]))


In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

print *, "---------"
! Bias in net fluxes of new result and RRTMGP, present-day
!print *, "#bias in net fluxes of new result and RRTMGP, ", &
!        "present-day: ", &
!        bias(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_nn(:,:,1), shape = [1*ncol*(nlay+1)])), &
!        bias(reshape(rldu_lbl(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_ref(:,:,1), shape = [1*ncol*(nlay+1)]))
! Bias in net fluxes of new result and RRTMGP, SURFACE
!print *, "#bias in net fluxes of new result and RRTMGP, ", &
!        "present-day, SURFACE: ", &
!        bias(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rldu_nn(nlay+1,:,1), shape = [1*ncol])), &
!        bias(reshape(rldu_lbl(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rldu_ref(nlay+1,:,1), shape = [1*ncol]))
! Bias in net fluxes of new result and RRTMGP, future
!print *, "#bias in net fluxes of new result and RRTMGP, ", &
!        "future: ", &
!        bias(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_nn(:,:,4), shape = [1*ncol*(nlay+1)])), &
!        bias(reshape(rldu_lbl(:,:,4), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_ref(:,:,4), shape = [1*ncol*(nlay+1)]))
! Bias in net fluxes of new result and RRTMGP, future-all
!print *, "#bias in net fluxes of new result and RRTMGP, ", &
!        "future-all: ", &
!        bias(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_nn(:,:,17), shape = [1*ncol*(nlay+1)])), &
!        bias(reshape(rldu_lbl(:,:,17), shape = [1*ncol*(nlay+1)]), &
!        reshape(rldu_ref(:,:,17), shape = [1*ncol*(nlay+1)]))
! Bias in net fluxes of new result and RRTMGP, SURFACE, future-all
print *, "bias in net fluxes of new result and RRTMGP, ", &
        "future-all, SURFACE: ", &
        bias(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
        reshape(rldu_nn(nlay+1,:,17), shape = [1*ncol])), &
        bias(reshape(rldu_lbl(nlay+1,:,17), shape = [1*ncol]), &
        reshape(rldu_ref(nlay+1,:,17), shape = [1*ncol]))
print *, "---------"
! Radiative forcing error at surface, pre-industrial N2O to present-day
print *, "radiative forcing error at surface, ", &
        "pre-industrial N2O to present-day: ", &
        mean(rld_lbl(nlay+1,:,11) - rld_lbl(nlay+1,:,1)) - &
        mean(rld_nn(nlay+1,:,11) - rld_nn(nlay+1,:,1)), &
        mean(rld_lbl(nlay+1,:,11) - rld_lbl(nlay+1,:,1)) - &
        mean(rld_ref(nlay+1,:,11) - rld_ref(nlay+1,:,1))
! Radiative forcing error at TOA, pre-industrial N2O to present-day
print *, "radiative forcing error at TOA, ", &
        "pre-industrial N2O to present-day: ", &
        mean(rlu_lbl(1,:,11) - rlu_lbl(1,:,1)) - &
        mean(rlu_nn(1,:,11) - rlu_nn(1,:,1)), &
        mean(rlu_lbl(1,:,11) - rlu_lbl(1,:,1)) - &
        mean(rlu_ref(1,:,11) - rlu_ref(1,:,1))
! MAE in upwelling fluxes of new result w.r.t RRTMGP, present-day
!print *, "#MAE in upwelling fluxes of new result w.r.t RRTMGP, ", &
!        "present-day: ", &
!        mae(reshape(rlu_ref(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rlu_nn(:,:,1), shape = [1*ncol*(nlay+1)]))
! MAE in upwelling fluxes of new result w.r.t RRTMGP, present-day, SFC
!print *, "#MAE in upwelling fluxes of new result w.r.t RRTMGP, ", &
!        "present-day, SFC: ", &
!        mae(reshape(rlu_ref(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rlu_nn(nlay+1,:,1), shape = [1*ncol]))
! MAE in downwelling fluxes of new result w.r.t RRTMGP, present-day, SFC
!print *, "#MAE in downwelling fluxes of new result w.r.t RRTMGP, ", &
!        "present-day, SFC: ", &
!        mae(reshape(rld_ref(nlay+1,:,1), shape = [1*ncol]), &
!        reshape(rld_nn(nlay+1,:,1), shape = [1*ncol]))
! MAE in downwelling fluxes of new result w.r.t RRTMGP, present-day
!print *, "#MAE in downwelling fluxes of new result w.r.t RRTMGP, ", &
!        "present-day: ", &
!        mae(reshape(rld_ref(:,:,1), shape = [1*ncol*(nlay+1)]), &
!        reshape(rld_nn(:,:,1), shape = [1*ncol*(nlay+1)]))

In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

print *, "---------"

! Print MAE for net fluxes with respect to RRTMGP
print *, "MAE in net flux w.r.t RRTMGP ", &
        mae(reshape(rldu_ref(:,:,:), shape = [nexp*ncol*(nlay+1)]), &
        reshape(rldu_nn(:,:,:), shape = [nexp*ncol*(nlay+1)]))

! Print max difference in downward flux with respect to RRTMGP
print *, "Max-diff in d.w. flux w.r.t RRTMGP ", &
        maxval(abs(rld_ref(:,:,:)-rld_nn(:,:,:)))

! Print max difference in upward flux with respect to RRTMGP
print *, "Max-diff in u.w. flux w.r.t RRTMGP ", &
        maxval(abs(rlu_ref(:,:,:)-rlu_nn(:,:,:)))

! Print max difference in net flux with respect to RRTMGP
print *, "Max-diff in net flux w.r.t RRTMGP ", &
        maxval(abs(rldu_ref(:,:,:)-rldu_nn(:,:,:)))

! Deallocate arrays
deallocate(rld_ref, rlu_ref, rld_nn, rlu_nn, rld_lbl, rlu_lbl, &
           rldu_ref, rldu_nn, rldu_lbl)

end if

In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

deallocate(flux_up, flux_dn)
print *, "SUCCESS!"

### Functions declaration part

In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

contains

function rmse(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: res
  real(wp), dimension(size(x1)) :: diff 
  
  diff = x1 - x2
  res = sqrt( sum(diff**2)/size(diff) )
end function rmse

function mae(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: res
  real(wp), dimension(size(x1)) :: diff 
  
  diff = abs(x1 - x2)
  res = sum(diff, dim=1)/size(diff, dim=1)
end function mae

function bias(x1,x2) result(res)
  implicit none 
  real(wp), dimension(:), intent(in) :: x1,x2
  real(wp) :: mean1,mean2, res
  
  mean1 = sum(x1, dim=1)/size(x1, dim=1)
  mean2 = sum(x2, dim=1)/size(x2, dim=1)
  res = mean1 - mean2

end function bias

function mean(x) result(mean1)
  implicit none 
  real(wp), dimension(:), intent(in) :: x
  real(wp) :: mean1
  mean1 = sum(x) / size(x)
end function mean

function mean_2d(x) result(mean2)
  implicit none 
  real(wp), dimension(:,:), intent(in) :: x
  real(wp) :: mean2
  mean2 = sum(x) / size(x)
end function mean_2d

function mean_3d(x) result(mean3)
  implicit none 
  real(wp), dimension(:,:,:), intent(in) :: x
  real(wp) :: mean3
  mean3 = sum(x) / size(x)
end function mean_3d

In [ ]:
%%writefile -a rrtmgp_rfmip_lw_02.f90

end program rrtmgp_rfmip_lw

#### End of program

---

Most of the cells below are based on the Notebook `ukk23test01-rfmip-clear-sky.ipynb`.

In [ ]:
# %%writefile -a rrtmgp_rfmip_lw_02.f90

In [ ]:
# %%writefile -a rrtmgp_rfmip_lw_02.f90

In [ ]:
# %%writefile test01.f90
# program test01
# implicit none
#   print *, "starting clear-sky LW computations, using neural &
#             networks as RRTMGP kernel"
# end program

In [ ]:
# ! gfortran test01.f90

In [ ]:
# ! ./a.out